In [1]:
import os
import pandas
import string
import nltk
import numpy as np

from tqdm.notebook import tqdm_notebook
from glob import glob
from nltk.corpus import stopwords

In [2]:
nltk.download('stopwords')
NLTK_STOP_WORDS = [word for word in stopwords.words('english')]

chars_to_remove = list(string.punctuation)
chars_to_remove += ["*", "-", "/", "+"]
chars_to_remove = set(chars_to_remove)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\m7irt\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
paths = glob(os.path.join('Original_Text_Files', '*.txt'))
print(f"Text Files Found: {len(paths)}")

Text Files Found: 22379


In [4]:
def remove_stop_words(input_file_path):
    output_file_path = "./Stopwords_Removed/" + os.path.basename(input_file_path)
    
    with open(input_file_path, "r") as input_file, open(output_file_path, "w+") as output_file:
        lines = input_file.readlines()
        for line in lines:
            words = line.split()
            for word in words:
                l_word = word.lower()
                if l_word in NLTK_STOP_WORDS:
                    words.remove(word)

            output_file.write(" ".join(words))
            output_file.write(" ")
    
    return output_file_path

In [5]:
def remove_punctuation(input_file_path):
    output_file_path = "./Punctuation_Removed/" + os.path.basename(input_file_path)
    
    with open(input_file_path, "r") as input_file, open(output_file_path, "w") as output_file:
        data = input_file.read()
        for char in chars_to_remove:
            data = data.replace(str(char), "")
        
        output_file.write(data)
    
    return output_file_path

In [6]:
def to_lowercase(input_file_path):
    output_file_path = "./Lowercased_Documents/" + os.path.basename(input_file_path)
    
    with open(input_file_path, "r") as input_file, open(output_file_path, "w+") as output_file:
        lines = input_file.readlines()
        for line in lines:
            words = line.split()
            lowercased_words = []
            for word in words:
                l_word = word.lower()
                lowercased_words.append(l_word)

            output_file.write(" ".join(lowercased_words))
            output_file.write(" ")

    return output_file_path

In [7]:
for input_path in tqdm_notebook(paths, desc = "Progress", ncols = 700, unit = " Files"):
    input_path = remove_stop_words(input_path)
    input_path = remove_punctuation(input_path)
    input_path = to_lowercase(input_path)

Progress:   0%|                                                                                               …

In [8]:
def get_document_content():
    docs_paths = glob(os.path.join('Lowercased_Documents', '*.txt'))
    docs_with_content = {}
    
    for input_path in tqdm_notebook(docs_paths, desc = "Progress", ncols = 700, unit = " Files"):
        doc_words = []
        with open(input_path, "r") as input_file:
            lines = input_file.readlines()
            for line in lines:
                words = line.split()
                for word in words:
                    doc_words.append(word)
        
        docs_with_content[os.path.basename(input_path)] = doc_words

    return docs_with_content

In [9]:
documents_content = get_document_content()

Progress:   0%|                                                                                               …

In [39]:
def create_vocabulary(documents):
    vocabulary = {}
    for doc_words in tqdm_notebook(documents.values(), desc = "Progress", ncols = 700, unit = " Documents"):
        for word in doc_words: 
            if word not in vocabulary:
                vocabulary[word] = True
    
    return vocabulary

In [40]:
vocabulary = create_vocabulary(documents_content)
print(f"Words in Vocabulary: {len(vocabulary)}")

Progress:   0%|                                                                                               …

Words in Vocabulary: 1808540


In [42]:
def get_documnet_term_frequency(documents_content):
    docs_paths = glob(os.path.join('Lowercased_Documents', '*.txt'))
    doc_term_frequency = {}
    
    for input_path in docs_paths:
        doc_name = os.path.basename(input_path)
        doc_term_frequency[doc_name] = {}
        
    for doc in tqdm_notebook(documents_content.keys(), desc = "Progress", ncols = 700, unit = " Documents"):
        current_doc_words = documents_content[doc]
        words_count = {}
        
        for word in current_doc_words:
            words_count[word] = 0
        
        for word in current_doc_words:
            words_count[word] += 1
        
        doc_term_frequency[doc] = words_count
    
    return doc_term_frequency

In [43]:
document_term_frequency = get_documnet_term_frequency(documents_content)

Progress:   0%|                                                                                               …

In [50]:
def get_word_document_frequency(vocabulary, document_tf):
    word_df = {}
    for vocab_word in tqdm_notebook(vocabulary.keys(), desc = "Progress", ncols = 700, unit = " Words"):
        word_doc_count = 0
        for tf in document_tf.values():
            if vocab_word in tf:
                word_doc_count += 1
                
        word_df[vocab_word] = word_doc_count
    return word_df

In [51]:
word_document_frequency = get_word_document_frequency(vocabulary, document_term_frequency)

Progress:   0%|                                                                                               …

In [54]:
def get_idf(vocab, m, word_df):
    inverse_df_vocab = {}
    for vocab_word in tqdm_notebook(vocab.keys(), desc = "Progress", ncols = 700, unit = " Words"):
        inverse_df_vocab[vocab_word] = np.log2((m + 1) / word_df[vocab_word])
    
    return inverse_df_vocab

In [55]:
idf_vocabulary = get_idf(vocabulary, len(paths), word_document_frequency)

Progress:   0%|                                                                                               …

In [61]:
def get_tf_idf(idf_vocab, doc_tf):
    tf_idf_docs = {}
    for doc_name in doc_tf.keys():
        tf_idf_docs[doc_name] = {}
        
    for (doc_name, words_freq) in tqdm_notebook(doc_tf.items(), desc = "Progress", ncols = 700, unit = " Documents"):
        tf_idf_words = {}
        for (doc_word, word_freq) in words_freq.items():
            tf_idf_words[doc_word] = word_freq * idf_vocab[doc_word]
            
        tf_idf_docs[doc_name] = tf_idf_words
        
    return tf_idf_docs

In [62]:
tf_idf_documents = get_tf_idf(idf_vocabulary, document_term_frequency)

Progress:   0%|                                                                                               …

### TF-IDF Weightage Sample Code
This is a sample code to give students an idea of how TF-IDF weightage model can be applied to calculate relevance score of documents.

The example is taken from **lectures**.

**P.S Do not get confused with the answers provided for TF-IDF in lecture 4-5, since the example shown had documents from d1-d5 but in actual the collection was large, hence M and k are actually unknown in that solved solution.** <br>
**In this example, M = 5 as it is assumed that the collection has 5 docs only.**

<h3 style = 'color:purple;'>Vector Space Model (TF-IDF Weightage Model)</h3>

$$ f(q,d) = sim(q,d) =  \sum_{i=1}^n x_iy_i $$ 
q = (x_1,.....,x_n) <br>
d = (y_1,.....,y_n) <br>
x_i = count of word W_i in query. <br>
y_i = TF-IDF of word W_i in doc i.e $$ y_i = C(W_i,doc) * log_2 \frac {M+1} {k} $$
M = number of documents in the collection <br>
k = document frequency for every word in corpus


In [ ]:
#lets say we have the following documents
documents = {
    "d1" : "news about",
    "d2" : "news about organic food campaign",
    "d3" : "news of presidential campaign",
    "d4" : "news of presidential campaign presidential candidate",
    "d5" : "news of organic food campaign campaign campaign campaign"
} # a dictionary with doc# as key and doc content as value

In [ ]:
#visualize the dictionary
documents

{'d1': 'news about',
 'd2': 'news about organic food campaign',
 'd3': 'news of presidential campaign',
 'd4': 'news of presidential campaign presidential candidate',
 'd5': 'news of organic food campaign campaign campaign campaign'}

In [ ]:
#create a corpus ccontaining the vocabulary of words in the documents
corpus = [] # a list that will store words of the vocabulary
for doc in documents.values(): #iterate through documents 
    for word in doc.split(): #go through each word in the current doc
        if not word in corpus: 
            corpus.append(word) #add word in corpus if not already added

In [ ]:
#visualize the corpus 
corpus

['news',
 'about',
 'organic',
 'food',
 'campaign',
 'of',
 'presidential',
 'candidate']

In [ ]:
#lets create a dictionary that will store document frequency for each word in the corpus
df_corpus = {} #document frequency for every word in corpus
for word in corpus:
    k = 0 #initial document frequency set to 0
    for doc in documents.values(): #iterate through documents
        if word in doc.split(): #check if word in doc
            k+=1 
    df_corpus[word] = k

In [ ]:
#verify the document frequency values
df_corpus

{'news': 5,
 'about': 2,
 'organic': 2,
 'food': 2,
 'campaign': 4,
 'of': 3,
 'presidential': 2,
 'candidate': 1}

In [ ]:
#since we have calculated k (document frequency) for all the words in the corpus, next step is to calculate idf
M = len(documents) #number of documents in the collection
idf_corpus = {} #inverse_document frequency for every word in corpus
for word in corpus:
    idf_corpus[word] = np.log2((M+1) / df_corpus[word]) #log_2 ((M+1)/k) i.e inverse document frequency

In [ ]:
#visualize idf values
idf_corpus

{'news': 0.2630344058337938,
 'about': 1.584962500721156,
 'organic': 1.584962500721156,
 'food': 1.584962500721156,
 'campaign': 0.5849625007211562,
 'of': 1.0,
 'presidential': 1.584962500721156,
 'candidate': 2.584962500721156}

We have successfully calculated inverse_document_frequency for all the words in the corpus. Now, using the idf values, we need to calculate tf-idf for each word with respect to a particular document.

In [ ]:
#calculating tf_idf
tf_idf_docs = {} #will store tf_idf scores for document words
for doc_id in documents.keys():
    tf_idf_docs[doc_id] = {} #initialize empty dictionary for each doc_id

In [ ]:
#visualize tf_idf initial state
tf_idf_docs

{'d1': {}, 'd2': {}, 'd3': {}, 'd4': {}, 'd5': {}}

In [ ]:
#finalizing the tf_idf calculations
for word in corpus:
    for doc_id,doc in documents.items(): #iterate through key,value pairs where key = doc_id and value = doc content
        tf_idf_docs[doc_id][word] = doc.split().count(word) * idf_corpus[word] #C(W_i,doc) * IDF(W_i) 

In [ ]:
#visualize final tf_idf scores for each doc
tf_idf_docs

{'d1': {'about': 1.584962500721156,
  'campaign': 0.0,
  'candidate': 0.0,
  'food': 0.0,
  'news': 0.2630344058337938,
  'of': 0.0,
  'organic': 0.0,
  'presidential': 0.0},
 'd2': {'about': 1.584962500721156,
  'campaign': 0.5849625007211562,
  'candidate': 0.0,
  'food': 1.584962500721156,
  'news': 0.2630344058337938,
  'of': 0.0,
  'organic': 1.584962500721156,
  'presidential': 0.0},
 'd3': {'about': 0.0,
  'campaign': 0.5849625007211562,
  'candidate': 0.0,
  'food': 0.0,
  'news': 0.2630344058337938,
  'of': 1.0,
  'organic': 0.0,
  'presidential': 1.584962500721156},
 'd4': {'about': 0.0,
  'campaign': 0.5849625007211562,
  'candidate': 2.584962500721156,
  'food': 0.0,
  'news': 0.2630344058337938,
  'of': 1.0,
  'organic': 0.0,
  'presidential': 3.169925001442312},
 'd5': {'about': 0.0,
  'campaign': 2.3398500028846247,
  'candidate': 0.0,
  'food': 1.584962500721156,
  'news': 0.2630344058337938,
  'of': 1.0,
  'organic': 1.584962500721156,
  'presidential': 0.0}}

### Querying the documents for relevance scores
Since we have calculated the term frequencies for all the documents in our collection, let us calcualte the relevance score of each document for a given query.

In [ ]:
query = "news about presidential campaign" #the query
query

'news about presidential campaign'

In [ ]:
query_vocab = [] # will store the unique words that occur in the query
for word in query.split():
    if word not in query_vocab:
        query_vocab.append(word)

In [ ]:
query_vocab # the unique words in the query

['news', 'about', 'presidential', 'campaign']

In [ ]:
query_wc = {} # a dictionary to store count of a word in the query (i.e x_i according to lecture slides terminology)
for word in query_vocab:
    query_wc[word] = query.split().count(word)

In [ ]:
query_wc # the count of each word that occurs in the query

{'about': 1, 'campaign': 1, 'news': 1, 'presidential': 1}

In [ ]:
relevance_scores = {} # a dictionary that will store the relevance score for each doc
# doc_id will be the key and relevance score the value for this dictionary
for doc_id in documents.keys():
    score = 0 #initialze the score for the doc to 0 at the start
    for word in query_vocab:
        score += query_wc[word] * tf_idf_docs[doc_id][word] # count of word in query * term_freq of the word
    relevance_scores[doc_id] = score

In [ ]:
# lets print the relevance scores for the query
print("Document Relevancy Scores\n",relevance_scores)

Document Relevancy Scores
 {'d1': 1.84799690655495, 'd2': 2.432959407276106, 'd3': 2.432959407276106, 'd4': 4.017921907997263, 'd5': 2.6028844087184186}
